In [23]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

def lerp(a, b, t):
    return a * (1-t) + b * t

def step(edge, x):
    return np.heaviside(x-edge, 0)

def smoothstep(edge0, edge1, x):
    t = np.clip((x - edge0) / (edge1 - edge0), 0.0, 1.0)
    return t * t * (3 - 2*t)

def smootherstep(edge0, edge1, x):
    # Scale, and clamp x to 0..1 range
    t = np.clip((x - edge0) / (edge1 - edge0), 0.0, 1.0)

    return t * t * t * (t * (6.0 * t - 15.0) + 10.0)


# The standard set of weight-space GP basis functions
class GPBasis():
    def __init__(self, l):
        self.thetas = np.random.normal(scale=1, size=l) # Spectral density of the squared exponential is the normal distribution
        self.taus = np.random.uniform(size=l) * 2 * np.pi # Uniformly distributed as in the paper

    def phi(self, x):
        print((self.thetas[None,:] * x[...,None]).shape)
        return np.sqrt(2/len(self.thetas)) * np.cos(self.thetas[None,:] * x[...,None] + self.taus)
    
    def size(self):
        return self.thetas.shape[0]



#Just another non-standard set of basis functions
class SinWaveBasis():
    def __init__(self, l):
        self.freq = np.linspace(0, 5, l)
        self.phase = np.linspace(0, np.pi*2, l)
        self.amplitude = 1/(2**np.linspace(0, 10, l))

    def phi(self, x):
        return np.sqrt(2/len(self.freq)) * np.sin(self.freq[None,:] * x[...,None] + self.phase)
    
    def size(self):
        return self.freq.shape[0]

# My interpretation of what "Perlin noise basis functions" would look like
# WJ: what's currently here is actually a combination of gradient noise (Perlin) and value noise
class PerlinNoiseBasis():
    def cell(self, x):
        interp = (x - self.x_range[0]) / (self.x_range[1] - self.x_range[0])
        cell_pos = interp * (self.random_numbers.shape[1] - 1)
        return np.array(np.floor(cell_pos), dtype=int), cell_pos - np.floor(cell_pos)

    def perlin(self, x, noise_idx):
        cell_x, frac_x = self.cell(x)
        cell_x = (cell_x) % self.random_numbers.shape[1]
        ncell_x = (cell_x+1) % self.random_numbers.shape[1]
        # this controls how much value noise is mixed in
        # setting this to 0 makes this true Perlin (gradient) noise
        value_amount = 0
        a = value_amount*self.random_numbers[noise_idx, cell_x, 0]
        b = value_amount*self.random_numbers[noise_idx, ncell_x, 0]

        # this controls how much gradient noise should be mixed it.
        # setting this to 0 (and value_amount to 1) creates value noise
        gradient_amount = 1
        a_x = gradient_amount*self.random_numbers[noise_idx, cell_x, 1]
        b_x = gradient_amount*self.random_numbers[noise_idx, ncell_x, 1]

        # perform linear extrapolation from neighboring grid locations
        a += a_x*frac_x
        b += b_x*(frac_x-1.0)

        # blend the extrapolated values
        # weight_b = step(0.5, frac_x)              # constant
        # weight_b = frac_x                         # linear
        weight_b = smootherstep(0.0,1.0,frac_x)     # cubic

        return a * (1-weight_b) + b * weight_b
    
    def size(self):
        return self.freq.shape[0] * self.random_numbers.shape[0]

    def __init__(self, n_octaves, n_noises, x_range=(0,20), grid_size=20):
        np.random.seed(51)
        self.x_range = x_range
        self.random_numbers = np.random.uniform(low=-1, high=1, size=(n_noises, grid_size+1, 2))
        self.freq = 2**np.linspace(0, n_octaves, n_octaves, False)
        # the division factor here just ensures that the largest scale octave has 4 wiggles over the x_range
        self.freq = self.freq / (x_range[1]-x_range[0]) * 4
        self.amplitude = 0.5**(np.linspace(0, n_octaves, n_octaves, False) + 1)
        self.phase = x_range[0] + np.random.uniform(size=n_octaves*n_noises) * (x_range[1] - x_range[0])


    def phi(self, x):
        noise_loc = self.freq[None,:] * (x[...,None])
        noise_scale = self.amplitude / np.sum(self.amplitude)
        noise_idx = np.tile(np.arange(0, self.random_numbers.shape[0]), noise_loc.shape)
        noise_loc = np.repeat(noise_loc, self.random_numbers.shape[0], axis=-1)
        noise_scale = np.repeat(noise_scale, self.random_numbers.shape[0], axis=-1)
        noise_idx = noise_idx.reshape(noise_loc.shape)

        return self.perlin(noise_loc + self.phase[None,...], noise_idx) * noise_scale

def cov(x,y):
    return np.exp(-(x-y)**2)

def apply_functional_update(x, weights, basis, cov, X, y, sigma):
    residual = y - evaluate_basis(X, basis, weights)
    cov_mat = cov(*np.meshgrid(X, X)) + sigma**2 * np.identity(X.shape[0])
    v = np.linalg.solve(cov_mat, residual)
    return evaluate_basis(x, basis, weights) + np.dot(v, cov(*np.meshgrid(x, X)))

# Evaluate Eq. (6)
def evaluate_basis(x, basis, weights):
    return np.sum(weights[None,...]*basis.phi(x), axis=-1)

# Compute mean and covariance of posterior weight distribution (Eq. (7))
def condition(X, y, basis, sigma):
    P = basis.phi(X)
    PtP = P.transpose() @ P
    cm_inv = np.linalg.inv(PtP + sigma**2 * np.identity(PtP.shape[0]))

    mu = cm_inv @ (P.transpose() @ y[...,None])
    Sigma = cm_inv * sigma**2
    return mu.flatten(), Sigma

# Number of basis functions
x_range = (0,20)

# Setup basis function
# basis = SinWaveBasis(l)
#basis = PerlinNoiseBasis(5, 10, x_range=x_range, grid_size=x_range[1])
basis = GPBasis(100)

# Evaluation points for plotting
xs = np.linspace(x_range[0], x_range[1], 2000)
ints = np.arange(x_range[0], x_range[1])

plt.title("Basis functions")
plt.plot(xs, basis.phi(xs))
# plt.plot(xs, np.zeros(2000))
# plt.plot(ints, basis.phi(ints), 'o')
plt.xticks(np.linspace(x_range[0], x_range[1], x_range[1]+1))
# plt.ylim(-1,1)
plt.show()


(2000, 100)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

def lerp(a, b, t):
    return a * (1-t) + b * t

def step(edge, x):
    return np.heaviside(x-edge, 0)

def smoothstep(edge0, edge1, x):
    t = np.clip((x - edge0) / (edge1 - edge0), 0.0, 1.0)
    return t * t * (3 - 2*t)

def smootherstep(edge0, edge1, x):
    # Scale, and clamp x to 0..1 range
    t = np.clip((x - edge0) / (edge1 - edge0), 0.0, 1.0)

    return t * t * t * (t * (6.0 * t - 15.0) + 10.0)


# The standard set of weight-space GP basis functions
class GPBasis():
    def __init__(self, l):
        self.thetas = np.random.normal(scale=1, size=l) # Spectral density of the squared exponential is the normal distribution
        self.taus = np.random.uniform(size=l) * 2 * np.pi # Uniformly distributed as in the paper

    def phi(self, x):
        return np.sqrt(2/len(self.thetas)) * np.cos(self.thetas[None,:] * x[...,None] + self.taus)
    
    def size(self):
        return self.thetas.shape[0]



#Just another non-standard set of basis functions
class SinWaveBasis():
    def __init__(self, l):
        self.freq = np.linspace(0, 5, l)
        self.phase = np.linspace(0, np.pi*2, l)
        self.amplitude = 1/(2**np.linspace(0, 10, l))

    def phi(self, x):
        return np.sqrt(2/len(self.freq)) * np.sin(self.freq[None,:] * x[...,None] + self.phase)
    
    def size(self):
        return self.freq.shape[0]

# My interpretation of what "Perlin noise basis functions" would look like
# WJ: what's currently here is actually a combination of gradient noise (Perlin) and value noise
class PerlinNoiseBasis():
    def cell(self, x):
        interp = (x - self.x_range[0]) / (self.x_range[1] - self.x_range[0])
        cell_pos = interp * (self.random_numbers.shape[1] - 1)
        return np.array(np.floor(cell_pos), dtype=int), cell_pos - np.floor(cell_pos)

    def perlin(self, x, noise_idx):
        cell_x, frac_x = self.cell(x)
        cell_x = (cell_x) % self.random_numbers.shape[1]
        ncell_x = (cell_x+1) % self.random_numbers.shape[1]
        # this controls how much value noise is mixed in
        # setting this to 0 makes this true Perlin (gradient) noise
        value_amount = 0
        a = value_amount*self.random_numbers[noise_idx, cell_x, 0]
        b = value_amount*self.random_numbers[noise_idx, ncell_x, 0]

        # this controls how much gradient noise should be mixed it.
        # setting this to 0 (and value_amount to 1) creates value noise
        gradient_amount = 1
        a_x = gradient_amount*self.random_numbers[noise_idx, cell_x, 1]
        b_x = gradient_amount*self.random_numbers[noise_idx, ncell_x, 1]

        # perform linear extrapolation from neighboring grid locations
        a += a_x*frac_x
        b += b_x*(frac_x-1.0)

        # blend the extrapolated values
        # weight_b = step(0.5, frac_x)              # constant
        # weight_b = frac_x                         # linear
        weight_b = smootherstep(0.0,1.0,frac_x)     # cubic

        return a * (1-weight_b) + b * weight_b
    
    def size(self):
        return self.freq.shape[0] * self.random_numbers.shape[0]

    def __init__(self, n_octaves, n_noises, x_range=(0,20), grid_size=20):
        np.random.seed(51)
        self.x_range = x_range
        self.random_numbers = np.random.uniform(low=-1, high=1, size=(n_noises, grid_size+1, 2))
        self.freq = 2**np.linspace(0, n_octaves, n_octaves, False)
        # the division factor here just ensures that the largest scale octave has 4 wiggles over the x_range
        self.freq = self.freq / (x_range[1]-x_range[0]) * 4
        self.amplitude = 0.5**(np.linspace(0, n_octaves, n_octaves, False) + 1)
        self.phase = x_range[0] + np.random.uniform(size=n_octaves*n_noises) * (x_range[1] - x_range[0])


    def phi(self, x):
        noise_loc = self.freq[None,:] * (x[...,None])
        noise_scale = self.amplitude / np.sum(self.amplitude)
        noise_idx = np.tile(np.arange(0, self.random_numbers.shape[0]), noise_loc.shape)
        noise_loc = np.repeat(noise_loc, self.random_numbers.shape[0], axis=-1)
        noise_scale = np.repeat(noise_scale, self.random_numbers.shape[0], axis=-1)
        noise_idx = noise_idx.reshape(noise_loc.shape)

        return self.perlin(noise_loc + self.phase[None,...], noise_idx) * noise_scale

def cov(x,y):
    return np.exp(-(x-y)**2)

def apply_functional_update(x, weights, basis, cov, X, y, sigma):
    residual = y - evaluate_basis(X, basis, weights)
    cov_mat = cov(*np.meshgrid(X, X)) + sigma**2 * np.identity(X.shape[0])
    v = np.linalg.solve(cov_mat, residual)
    return evaluate_basis(x, basis, weights) + np.dot(v, cov(*np.meshgrid(x, X)))

# Evaluate Eq. (6)
def evaluate_basis(x, basis, weights):
    return np.sum(weights[None,...]*basis.phi(x), axis=-1)

# Compute mean and covariance of posterior weight distribution (Eq. (7))
def condition(X, y, basis, sigma):
    P = basis.phi(X)
    PtP = P.transpose() @ P
    cm_inv = np.linalg.inv(PtP + sigma**2 * np.identity(PtP.shape[0]))

    mu = cm_inv @ (P.transpose() @ y[...,None])
    Sigma = cm_inv * sigma**2
    return mu.flatten(), Sigma

# Number of basis functions
x_range = (0,20)

# Setup basis function
# basis = SinWaveBasis(l)
#basis = PerlinNoiseBasis(5, 10, x_range=x_range, grid_size=x_range[1])
basis = GPBasis(100)

# Evaluation points for plotting
xs = np.linspace(x_range[0], x_range[1], 2000)
ints = np.arange(x_range[0], x_range[1])

plt.title("Basis functions")
plt.plot(xs, basis.phi(xs))
# plt.plot(xs, np.zeros(2000))
# plt.plot(ints, basis.phi(ints), 'o')
plt.xticks(np.linspace(x_range[0], x_range[1], x_range[1]+1))
# plt.ylim(-1,1)
plt.show()


In [19]:

plt.title("Adding up realizations, equal weight")
# Sample and plot 50 realizations
real = evaluate_basis(xs,basis,np.ones(basis.size()))
plt.plot(xs, real, color="b")

plt.show()


In [20]:

plt.title("Sample some prior realizations")
weights = np.random.normal(size=(5,basis.size()))
for w in weights:
    real = evaluate_basis(xs,basis,w)
    plt.plot(xs, real, alpha=0.7)

plt.show()


In [21]:
from matplotlib.animation import FuncAnimation
from matplotlib import cm

fig = plt.figure(figsize=(8,4))
num_iters = 20
colors = cm.coolwarm(np.linspace(0, 1, num_iters))

def animate(i):
  # Condition locations and values
  cond_x = np.array([0, 3, 9.65+i*0.03, 15, 20])
  cond_y = np.array([0, 1, -1+i*0.1, -1, 0])

  # Compute posterior weight mean and covariances
  cond = condition(cond_x, cond_y, basis, 0.01)
  
  # Sample and plot 1 realization
  weights_conditioned = np.atleast_2d(multivariate_normal(mean=cond[0], cov=cond[1], allow_singular=True).rvs(size=1,random_state=1))
  weights_unconditioned = np.atleast_2d(multivariate_normal(mean=np.zeros(basis.size()), cov=np.identity(basis.size())).rvs(size=1,random_state=1))
  
  for wi in range(weights_conditioned.shape[0]):
      real_distributional = evaluate_basis(xs,basis,weights_conditioned[wi])
      real_functional = apply_functional_update(xs, weights_unconditioned[wi], basis, cov, cond_x, cond_y, 0.01)
      plt.plot(xs, real_distributional, alpha=0.2, color=colors[i])
      plt.plot(xs, real_functional, ":", alpha=0.2, color=colors[i])

  # Draw conditioning points
  plt.scatter(cond_x, cond_y, color=colors[i], zorder=100)

# ani = FuncAnimation(fig, animate, interval=4)
# ani.save('test.gif')
plt.title("Sampled posterior realizations")
for i in range(num_iters):
    animate(i)
plt.show()

In [22]:
import math

import matplotlib.pyplot as plt
from matplotlib.backend_bases import MouseEvent
from matplotlib import cm
from matplotlib.widgets import Slider, Button
%matplotlib qt

class DraggablePlotExample(object):
    u""" An example of plot with draggable markers """

    def __init__(self):
        self._figure, self._axes, self._line_dist, self._line_func, self._scatter = None, None, None, None, None
        self._dragging_point = None
        self._points = {}
        self._colors = cm.coolwarm(np.linspace(0, 1, num_iters))

        self._init_plot()

    def _init_plot(self):
        self._figure = plt.figure("Conditioning Editor")
        axes = plt.subplot(1, 1, 1)
        axes.set_xlim(0, 20)
        axes.set_ylim(-5.5, 5.5)
        axes.grid(which="both")
        self._axes = axes

        self._figure.subplots_adjust(bottom=0.25)

        self._figure.canvas.mpl_connect('button_press_event', self._on_click)
        self._figure.canvas.mpl_connect('button_release_event', self._on_release)
        self._figure.canvas.mpl_connect('motion_notify_event', self._on_motion)
        self._axsigma = self._figure.add_axes([0.1, 0.1, 0.8, 0.03])
        self._sigma_slider = Slider(
            ax=self._axsigma,
            label='$\\sigma$',
            valmin=0.0001,
            valmax=1,
            valinit=0.1,
        )

        self._sigma_slider.on_changed(lambda _: self._update_plot())

        plt.show()

    def _update_plot(self):
        if not self._points:
            self._line_dist.set_data([], [])
            self._line_func.set_data([], [])
        else:
            # Condition locations and values
            x, y = zip(*sorted(self._points.items()))

            cond_x = np.array(x)
            cond_y = np.array(y)

            # Compute posterior weight mean and covariances
            cond = condition(cond_x, cond_y, basis, self._sigma_slider.val)

            weights_conditioned = np.atleast_2d(multivariate_normal(mean=cond[0], cov=cond[1], allow_singular=True).rvs(size=1,random_state=1))
            weights_unconditioned = np.atleast_2d(multivariate_normal(mean=np.zeros(basis.size()), cov=np.identity(basis.size())).rvs(size=1,random_state=1))
            
            for wi in range(weights_conditioned.shape[0]):
                real_distributional = evaluate_basis(xs,basis,weights_conditioned[wi])
                real_functional = apply_functional_update(xs, weights_unconditioned[wi], basis, cov, cond_x, cond_y, self._sigma_slider.val)
                if not self._line_dist:
                    self._line_dist = self._axes.plot(xs, real_distributional, color=self._colors[0], label="distributional")[0]
                    self._line_func = self._axes.plot(xs, real_functional, ":", color=self._colors[0], label="functional")[0]
                else:
                    self._line_dist.set_data(xs, real_distributional)
                    self._line_func.set_data(xs, real_functional)

            # Draw conditioning points
            if not self._scatter:
                self._axes.legend()
                self._scatter = self._axes.scatter(cond_x, cond_y, color=self._colors[-1], zorder=100)
            else:
                self._scatter.set_offsets(np.c_[cond_x, cond_y])
                
        self._figure.canvas.draw()

    def _add_point(self, x, y=None):
        if isinstance(x, MouseEvent):
            x, y = x.xdata, x.ydata
        self._points[x] = y
        return x, y

    def _remove_point(self, x, _):
        if x in self._points:
            self._points.pop(x)

    def _find_neighbor_point(self, event):
        u""" Find point around mouse position

        :rtype: ((int, int)|None)
        :return: (x, y) if there are any point around mouse else None
        """
        distance_threshold = 2
        nearest_point = None
        min_distance = math.sqrt(2 * (20 ** 2))
        for x, y in self._points.items():
            distance = math.hypot(event.xdata - x, event.ydata - y)
            if distance < min_distance:
                min_distance = distance
                nearest_point = (x, y)
        if min_distance < distance_threshold:
            return nearest_point
        return None

    def _on_click(self, event):
        u""" callback method for mouse click event

        :type event: MouseEvent
        """
        # left click
        if event.button == 1 and event.inaxes in [self._axes]:
            point = self._find_neighbor_point(event)
            if point:
                self._dragging_point = point
            else:
                self._add_point(event)
            self._update_plot()
        # right click
        elif event.button == 3 and event.inaxes in [self._axes]:
            point = self._find_neighbor_point(event)
            if point:
                self._remove_point(*point)
                self._update_plot()

    def _on_release(self, event):
        u""" callback method for mouse release event

        :type event: MouseEvent
        """
        if event.button == 1 and event.inaxes in [self._axes] and self._dragging_point:
            self._dragging_point = None
            self._update_plot()

    def _on_motion(self, event):
        u""" callback method for mouse motion event

        :type event: MouseEvent
        """
        if not self._dragging_point:
            return
        if event.xdata is None or event.ydata is None:
            return
        self._remove_point(*self._dragging_point)
        self._dragging_point = self._add_point(event)
        self._update_plot()

plot = DraggablePlotExample()